In [1]:
library(dplyr)
library(tidyr)
library(tibble)
library(qvalue)
library(data.table)

source("/mnt/lareaulab/reliscu/code/fisher_test.R")

setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




Here I perform enrichment analysis to find modules enriched for cell type markers. These modules will later be used to correlate to exon PSI to find cell type-specific exons.

In [2]:
network_dir <- "mouse_ACA_20pcntCells_0.4pcntVar_200samples_log2_Modules"

In [3]:
get_module_enrichments <- function(network_dir, ctype_genes, mod_def="PosFDR") {
    # Traverse networks to get cell type enrichments for each module
    networks <- list.dirs(file.path(getwd(), network_dir), full.names=TRUE, recursive=FALSE)
    networks <- networks[lengths(lapply(networks, list.files)) > 0]

    enrichments_list <- lapply(seq_along(networks), function(i) {
        kME_path <- list.files(networks[i])[grep("kME", list.files(networks[i]))]
        kME <- fread(file.path(networks[i], kME_path), data.table=FALSE)
        mod_col <- grep("PosFDR", colnames(kME))
        mod_genes <- tapply(kME$Gene, kME[,mod_col], list)
        
        if (length(mod_genes) > 0) {
            all_genes <- kME$Gene

            # For each module: calculate enrichment for DE genes from each cell type
            mod_enrichments_list <- lapply(mod_genes, function(mod) {
                lapply(unlist(lapply(ctype_genes, function(set) {
                    fisher_test(set, mod, all=all_genes)
                })), c)
            })
            
            # Save the network module came from
            network_id <- sapply(strsplit(networks[i], "/"), function(x) x[length(x)])
            mod_enrichments_df <- reshape2::melt(mod_enrichments_list)
            colnames(mod_enrichments_df) <- c("Pval", "Cell_type", "Module")
            
            # Save path to module eigengenes table for downstream analyses
            ME_path <- list.files(networks[i])[grep("eigengene", list.files(networks[i]))]
            
            data.frame(
                Network=network_id,
                kME_path=file.path(networks[i], kME_path),
                ME_path=file.path(networks[i], ME_path),
                mod_enrichments_df
            )
        }
    })
    enrichments_df <- do.call(rbind, enrichments_list)
    enrichments_df$Qval <- qvalue(enrichments_df$Pval)$qvalue

    enrichments_df
}

### Get DE genes frome each cell type: 1 vs. pooled tests

In [4]:
# pooled_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream.RDS")

In [ ]:
pval_threshold <- .05/length(pooled_res_list)
lfc_threshold <- 6

ctype_genes <- lapply(mean_res_list, function(df) {
    mask <- (df['adj.P.Val'] < pval_threshold) & (abs(df['logFC']) > lfc_threshold)
    df[mask, 1]
})
names(ctype_genes) <- names(mean_res_list) 

In [ ]:
data.frame(No.genes=lengths(ctype_genes)) %>% arrange(-`No.genes`)

In [ ]:
enrichments_df <- get_module_enrichments(network_dir, ctype_genes)

In [ ]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pooled_top_mods_df <- enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [ ]:
pooled_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

### Get DE genes frome each cell type: pairwise tests

In [10]:
pairwise_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream_processed.RDS")

In [59]:
ctypes <- unique(sapply(strsplit(names(pairwise_res_list), "_vs_"), "[", 1))

ctype_genes <- lapply(ctypes, function(target) {
    ctype_res_list <- pairwise_res_list[grep(paste0("^", target), names(pairwise_res_list))]
    pval_threshold <- .05 /length(ctype_res_list)
    ctype_genes_list <- lapply(ctype_res_list, function(df) {
        mask <- df$adj.P.Val < pval_threshold
        df[mask, 1]
    }) 
    Reduce(intersect, ctype_genes_list)
})
names(ctype_genes) <- ctypes

In [60]:
data.frame(No.genes=lengths(ctype_genes)) %>% arrange(-`No.genes`)

,No.genes
,<int>
Macrophage,333
Astro,288
Endo,237
Oligo,131
VLMC,85
SMC,60
Pvalb,31
Lamp5,22
CR,21


In [61]:
ctype_genes <- ctype_genes[lengths(ctype_genes) > 0]

In [62]:
enrichments_df <- get_module_enrichments(network_dir, ctype_genes)

In [63]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pairwise_top_mods_df <- enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    # filter(Qval < .05) %>%
    arrange(Qval)

In [64]:
pairwise_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Astro,0.000000e+00,0.000000e+00,turquoise,Bicor-None_signum0.219_minSize10_merge_ME_0.9_20151
Endo,0.000000e+00,0.000000e+00,blue,Bicor-None_signum0.219_minSize12_merge_ME_0.9_20151
Macrophage,0.000000e+00,0.000000e+00,blue,Bicor-None_signum0.794_minSize10_merge_ME_0.9_20151
Oligo,2.121824e-204,1.856647e-201,green,Bicor-None_signum0.325_minSize10_merge_ME_0.9_20151
VLMC,4.271020e-139,2.853896e-136,tan,Bicor-None_signum0.794_minSize3_merge_ME_0.9_20151
SMC,1.527776e-95,9.516493e-93,black,Bicor-None_signum0.794_minSize6_merge_ME_0.9_20151
NP,5.714908e-27,2.258372e-24,brown,Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151
Pvalb,7.313579e-27,2.874667e-24,midnightblue,Bicor-None_signum0.325_minSize12_merge_ME_0.9_20151
Sncg,1.426546e-22,5.016938e-20,brown2,Bicor-None_signum0.255_minSize10_merge_ME_0.9_20151


### Get DE genes frome each cell type: 1 vs. mean analysis

In [65]:
mean_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream.RDS")

In [66]:
pval_threshold <- .05/length(mean_res_list)
lfc_threshold <- 6

ctype_genes <- lapply(mean_res_list, function(df) {
    mask <- (df['adj.P.Val'] < pval_threshold) & (abs(df['logFC']) > lfc_threshold)
    df[mask, 1]
})
names(ctype_genes) <- names(mean_res_list) 

In [67]:
data.frame(No.genes=lengths(ctype_genes)) %>% arrange(-`No.genes`)

,No.genes
,<int>
Peri,1806
Endo,879
Macrophage,828
VLMC,551
SMC,542
Astro,503
Oligo,228
Vip,67
Sncg,66


In [68]:
enrichments_df <- get_module_enrichments(network_dir, ctype_genes)

In [69]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

mean_top_mods_df <- enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    filter(Qval < .05) %>%
    arrange(Qval)

In [70]:
mean_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Astro,1.682590e-233,1.292953e-228,turquoise,Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151
Macrophage,3.766146e-207,3.617524e-203,yellow,Bicor-None_signum0.919_minSize6_merge_ME_0.9_20151
Endo,2.400244e-187,1.317443e-183,blue,Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151
Oligo,4.937472e-150,6.219839e-147,red,Bicor-None_signum0.794_minSize12_merge_ME_0.9_20151
SMC,3.267160e-132,2.670834e-129,brown,Bicor-None_signum0.919_minSize6_merge_ME_0.9_20151
VLMC,2.575901e-114,1.783243e-111,tan,Bicor-None_signum0.794_minSize3_merge_ME_0.9_20151
Peri,2.195836e-83,1.349877e-80,white,Bicor-None_signum0.586_minSize8_merge_ME_0.9_20151
NP,9.227342e-56,4.431604e-53,brown,Bicor-None_signum0.325_minSize4_merge_ME_0.9_20151
Sncg,1.764350e-54,8.369009e-52,palevioletred3,Bicor-None_signum0.325_minSize10_merge_ME_0.9_20151
